In [ ]:
!pip install -q torch torchvision transformers pillow pandas scikit-learn

In [ ]:
ZIP_PATH = "/content/mvsa.zip"   # change if filename is different

!mkdir -p /content/mvsa_raw
!unzip -o "$ZIP_PATH" -d /content/mvsa_raw

!ls -R /content/mvsa_raw

In [ ]:
import os
from pathlib import Path
import pandas as pd

ROOT_DIR   = "/content/mvsa_raw/MVSA_Single"
DATA_DIR   = os.path.join(ROOT_DIR, "data")
LABEL_FILE = os.path.join(ROOT_DIR, "labelResultAll.txt")

print("ROOT_DIR :", ROOT_DIR)
print("DATA_DIR :", DATA_DIR)
print("LABEL_FILE:", LABEL_FILE)

# 1) Read labelResultAll.txt
# First column: ID
# Second column header: "text,image"
labels_raw = pd.read_csv(LABEL_FILE, sep=r"\s+", engine="python")
print(labels_raw.head())
print("Columns:", labels_raw.columns)

# 2) Split "text,image" into two sentiment columns
sent_col = labels_raw.columns[1]  # should be "text,image"
labels_raw[["text_sent", "image_sent"]] = labels_raw[sent_col].str.split(",", expand=True)

# 3) Use text sentiment as label
labels = labels_raw[["ID", "text_sent"]].copy()
labels["ID"] = labels["ID"].astype(int)
labels["text_sent"] = labels["text_sent"].str.lower().str.strip()

sent_to_id = {"negative": 0, "neutral": 1, "positive": 2}
labels["label"] = labels["text_sent"].map(sent_to_id)

print("Sentiment mapping:", sent_to_id)
print(labels.head())

In [ ]:
rows = []

for _, row in labels.iterrows():
    id_int = row["ID"]
    id_str = str(id_int)

    img_path = os.path.join(DATA_DIR, f"{id_str}.jpg")
    txt_path = os.path.join(DATA_DIR, f"{id_str}.txt")

    # ensure both files exist
    if not (os.path.exists(img_path) and os.path.exists(txt_path)):
        continue

    # read text
    with open(txt_path, "r", encoding="utf-8", errors="ignore") as f:
        text = f.read().strip()

    if not text:
        continue

    rows.append({
        "id": id_int,
        "text": text,
        "image_path": img_path,
        "label": row["label"],
    })

df = pd.DataFrame(rows)
print(df.head())
print("Total usable samples:", len(df))

# Optional: subsample for quicker training
N_SAMPLES = 4000  # adjust as you like
if len(df) > N_SAMPLES:
    df = df.sample(N_SAMPLES, random_state=42).reset_index(drop=True)

print("Using samples:", len(df))
df["label"].value_counts()

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
MAX_LEN = 64

# Image transforms
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1,
                           saturation=0.1, hue=0.02),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

class MVSASingleDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # text → tokens
        text = str(row["text"])
        enc = tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=MAX_LEN,
            return_tensors="pt"
        )

        # image
        image = Image.open(row["image_path"]).convert("RGB")
        image = self.transform(image)

        label = torch.tensor(row["label"], dtype=torch.long)

        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "image": image,
            "label": label
        }

# Stratified split
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

train_dataset = MVSASingleDataset(train_df, transform=train_transform)
val_dataset   = MVSASingleDataset(val_df,   transform=val_transform)

BATCH_SIZE = 16

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False)

len(train_loader), len(val_loader)

In [ ]:
import torch.nn as nn
from transformers import AutoModel
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

class MultimodalSentimentNet(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()

        # Text encoder
        self.text_encoder = AutoModel.from_pretrained("bert-base-uncased")
        text_dim = self.text_encoder.config.hidden_size  # 768

        # Image encoder
        resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        img_dim = resnet.fc.in_features                  # 512
        resnet.fc = nn.Identity()
        self.image_encoder = resnet

        # Fusion classifier
        fusion_dim = text_dim + img_dim  # 768 + 512
        self.classifier = nn.Sequential(
            nn.Linear(fusion_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes)
        )

    def forward(self, input_ids, attention_mask, images):
        # Text branch
        text_out = self.text_encoder(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        text_emb = text_out.last_hidden_state[:, 0, :]  # CLS token

        # Image branch
        img_emb = self.image_encoder(images)

        # Fuse
        fused = torch.cat([text_emb, img_emb], dim=1)
        logits = self.classifier(fused)
        return logits

NUM_CLASSES = 3  # negative / neutral / positive
model = MultimodalSentimentNet(num_classes=NUM_CLASSES).to(device)

In [ ]:
import torch.nn as nn
from torch.optim import AdamW

# Class weights (handle imbalance)
class_counts = train_df["label"].value_counts().sort_index().values.astype(np.float32)
inv = 1.0 / class_counts
weights = inv / inv.sum() * len(class_counts)

class_weights = torch.tensor(weights, dtype=torch.float32).to(device)
print("Class counts :", class_counts)
print("Class weights:", class_weights)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)

In [ ]:
from torch.nn.functional import cross_entropy

def run_epoch(loader, train=True):
    if train:
        model.train()
    else:
        model.eval()

    total_loss = 0.0
    correct = 0
    total = 0

    for batch in loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        images = batch["image"].to(device)
        labels = batch["label"].to(device)

        if train:
            optimizer.zero_grad()

        with torch.set_grad_enabled(train):
            logits = model(input_ids, attention_mask, images)
            loss = criterion(logits, labels)

            if train:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    acc = correct / total
    return avg_loss, acc


EPOCHS = 15
best_val_acc = 0.0
patience = 3
patience_counter = 0
best_path = "/content/mvsa_multimodal_best.pt"

for epoch in range(1, EPOCHS + 1):
    train_loss, train_acc = run_epoch(train_loader, train=True)
    val_loss, val_acc = run_epoch(val_loader, train=False)

    print(f"Epoch {epoch:02d}: "
          f"train_loss={train_loss:.4f}, train_acc={train_acc:.3f}, "
          f"val_loss={val_loss:.4f}, val_acc={val_acc:.3f}")

    # Early stopping on val_acc
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        torch.save(model.state_dict(), best_path)
        print(f"  🔸 New best model saved (val_acc={best_val_acc:.3f})")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("  🔻 Early stopping triggered.")
            break

print("Best validation accuracy:", best_val_acc)

In [ ]:
model.load_state_dict(torch.load(best_path, map_location=device))
model.eval()

FINAL_SAVE = "/content/mvsa_multimodal_sentiment.pt"
torch.save(model.state_dict(), FINAL_SAVE)
print("Final best model saved at:", FINAL_SAVE)